In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV


import tensorflow as tf
from tensorflow import keras
from hyperopt import fmin, tpe, hp
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


from sklearn.ensemble import AdaBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_classification
from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


import joblib
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score

In [11]:
df = pd.read_csv('dataset.csv')
df.rename(columns = {'Reached.on.Time_Y.N' : 'Reached'}, inplace = True)
df = df.drop('ID', axis = 1)
df.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached
0,D,Flight,4,2,177,3,low,F,44,1233,1
1,F,Flight,4,5,216,2,low,M,59,3088,1
2,A,Flight,2,2,183,4,low,M,48,3374,1
3,B,Flight,3,3,176,4,medium,M,10,1177,1
4,C,Flight,2,2,184,3,medium,F,46,2484,1


In [12]:
df['Reached'].unique()

array([1, 0])

In [13]:
df1 = pd.get_dummies(df, columns=['Warehouse_block', 'Mode_of_Shipment'], dtype = int)
df1.drop(['Mode_of_Shipment_Ship', 'Warehouse_block_F'], axis = 1, inplace = True)
df1['Product_importance'] = df1['Product_importance'].apply(lambda x : 3 if x == 'high' else (2 if x == 'medium' else 1))
df1['Gender'] = df1['Gender'].apply(lambda x : 1 if x == 'M' else 0)
df1


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Mode_of_Shipment_Flight,Mode_of_Shipment_Road
0,4,2,177,3,1,0,44,1233,1,0,0,0,1,1,0
1,4,5,216,2,1,1,59,3088,1,0,0,0,0,1,0
2,2,2,183,4,1,1,48,3374,1,1,0,0,0,1,0
3,3,3,176,4,2,1,10,1177,1,0,1,0,0,1,0
4,2,2,184,3,2,0,46,2484,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,4,1,252,5,2,0,1,1538,1,1,0,0,0,0,0
10995,4,1,232,5,2,0,6,1247,0,0,1,0,0,0,0
10996,5,4,242,5,1,0,4,1155,0,0,0,1,0,0,0
10997,5,2,223,6,2,1,2,1210,0,0,0,0,0,0,0


In [14]:
df1['Reached'].unique()

array([1, 0])

In [20]:
for column in df1.columns:
    print('='*20)
    print(column, " : ")
    print(df1[column].unique())

Customer_care_calls  : 
[4 2 3 5 6 7]
Customer_rating  : 
[2 5 3 1 4]
Cost_of_the_Product  : 
[177 216 183 176 184 162 250 233 150 164 189 232 198 275 152 227 143 239
 145 161 156 211 251 225 172 234 266 257 223 149 137 181 215 269 139 174
 151 210 169 160 190 141 165 170 203 246 238 193 221 179 105 261 202 109
 158 231 206 187 230 113 180 132 217 197 185 278 229 186 286 175 219 213
 235 271 144 218 263 168 265 205 252 222 220 147 200 224 247 280 157 207
 237 264 248 191 146 135  98  97 114 112 274 166 148 270 242 192 116 255
 209 134 130 133 140 136 142 154 155 127 129 159 294 226 258 241 208 182
 115 212 171 249 243 163 272 138 273 279 173 194 262 201 260 188 267 131
 122 103 199 236 167 259 178 123 124  96 244 254 128 204 245 228 268 108
 276 214 281 253 104 240 121 153 111 117 195 110 119 196 291 118 283 100
 256 285 284 101 296 277 106 282 126 102 120  99 125 107 301 290 310 308
 300 303 306 292 293 295 304 298 305 287 309 302 307 289 297 299 288]
Prior_purchases  : 
[ 3  2  4  6 

In [15]:
X = df1.drop('Reached', axis = 1)
y = df1['Reached']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with a StandardScaler and an SVM classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('SVM', SVC(C=0.001, gamma=0.01, kernel='rbf', probability=False, tol=0.001))
])

# Train the SVM model on the training data
pipeline.fit(X_train, y_train)

# Save the pipeline as a new pickle file
new_pipeline_filename = 'SVM_pipeline_new.pkl'
joblib.dump(pipeline, new_pipeline_filename)
print(f'Saved new SVM pipeline to {new_pipeline_filename}')

Saved new SVM pipeline to SVM_pipeline_new.pkl


In [21]:
try:
    pipeline = joblib.load('SVM_pipeline_new.pkl')  # Replace with the correct pickle file path
except Exception as e:
    print(f"Error loading the pipeline: {str(e)}")

# Check if the loaded object is a valid pipeline
if pipeline is not None and hasattr(pipeline, 'predict'):
    # Prepare input data for prediction (replace with your data)
    input_data = pd.DataFrame(columns=['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
                                    'Prior_purchases', 'Product_importance', 'Gender', 'Discount_offered',
                                    'Weight_in_gms', 'Warehouse_block_A', 'Warehouse_block_B',
                                    'Warehouse_block_C', 'Warehouse_block_D', 'Mode_of_Shipment_Flight',
                                    'Mode_of_Shipment_Road'],
                            data=np.array([2, 4, 435, 2, 1, 1, 20, 200, 1, 0, 0, 0, 0, 1]).reshape(1, 14))

    prediction = pipeline.predict(input_data)
    print(prediction)

    # Print the prediction (0 or 1 for binary classification)
    print("Prediction:", prediction[0])
else:
    print("Loaded object is not a valid pipeline.")

[1]
Prediction: 1


In [2]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [74]:
y

0        1
1        1
2        1
3        1
4        1
        ..
10994    1
10995    0
10996    0
10997    0
10998    0
Name: Reached, Length: 10999, dtype: int64

In [96]:
import joblib
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score


class ModelTrainer:
    def __init__(self, model, param_grid):
        self.model = model
        self.param_grid = param_grid
        self.best_model = None

    def train(self, X_train, y_train):
        grid_search = RandomizedSearchCV(self.model, self.param_grid, scoring='f1', cv=5, n_iter=10, random_state=42)
        grid_search.fit(X_train, y_train)
        self.best_model = grid_search.best_estimator_
        return self.best_model

    def predict(self, X_test):
        return self.best_model.predict(X_test)

def main(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Use y_binary

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define models and hyperparameter grids
    models = {
        'Logistic Regression': (LogisticRegression(),
                                {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2']}),
        
        'SVM': (SVC(),
                {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'],
                 'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1]})
    }

    best_models = {}
    
    for model_name, (model, param_grid) in models.items():
        try:
            trainer = ModelTrainer(model, param_grid)
            best_model = trainer.train(X_train_scaled, y_train)
            y_pred = trainer.predict(X_test_scaled)
            f1 = f1_score(y_test, y_pred, average='binary')  # Use 'binary' for binary classification
            print(f'{model_name} - Best F1 Score: {f1:.4f} - Best Hyperparameters: {trainer.best_model.get_params()}')
        
            best_models[model_name] = trainer.best_model

            # Create a pipeline for the best model
            pipeline = Pipeline([
                ('scaler', scaler),
                (model_name, trainer.best_model)
            ])

            # Save the pipeline as a pickle file
            pipeline_filename = f'{model_name}_pipeline.pkl'
            joblib.dump(pipeline, pipeline_filename)
            print(f'Saved {model_name} pipeline to {pipeline_filename}')
        except Exception as e:
            print(f"Error in model '{model_name}': {str(e)}")

    if best_models:
        best_model_name = max(best_models, key=lambda k: f1_score(y_test, best_models[k].predict(X_test_scaled)))
        best_model = best_models[best_model_name]
        print(f'Best Model: {best_model_name} - Best F1 Score: {f1:.4f}')
    else:
        print("No models successfully trained.")

if __name__ == "__main__":
    main(X, y)


/Users/rachitkumarsingh/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Logistic Regression - Best F1 Score: 0.7246 - Best Hyperparameters: {'C': 0.001, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Saved Logistic Regression pipeline to Logistic Regression_pipeline.pkl
SVM - Best F1 Score: 0.7447 - Best Hyperparameters: {'C': 0.001, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Saved SVM pipeline to SVM_pipeline.pkl
Best Model: SVM - Best F1 Score: 0.7447


In [97]:
import pickle
model=pickle.load(open('Logistic Regression_pipeline.pkl','rb'))

In [99]:
len(X.columns)

14

In [102]:
import pickle
import pandas as pd
import numpy as np

# Load the model or pipeline from the pickle file
model = pickle.load(open('Logistic Regression_pipeline.pkl', 'rb'))  # Replace with the correct pickle file name

# Prepare input data for prediction
input_data = pd.DataFrame(columns=['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
                                   'Prior_purchases', 'Product_importance', 'Gender', 'Discount_offered',
                                   'Weight_in_gms', 'Warehouse_block_A', 'Warehouse_block_B',
                                   'Warehouse_block_C', 'Warehouse_block_D', 'Mode_of_Shipment_Flight',
                                   'Mode_of_Shipment_Road'],
                          data=np.array([2, 4, 435, 2, 1, 1, 20, 200, 1, 0, 0, 0, 0, 1]).reshape(1, 14))

# Make predictions
prediction = model.predict(input_data)

print(prediction)


AttributeError: 'numpy.ndarray' object has no attribute 'predict'

In [ ]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))